In [ ]:
import os
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import cv2

# you should also visualize this shit to sanity-test it

In [ ]:
def get_ra_dfs():

    """not pretty but gets the job done..."""

    tuple_dir = "/home/simon/Documents/Bodies/data/ra_outputs"

    score_dict = {}

    for root, dirs, files in os.walk(tuple_dir):
        for tuple_name in files:
            if tuple_name.split('_')[-1] == 'tuple.pkl':
                tuple_path = os.path.join(tuple_dir, tuple_name) 
        
                with open(tuple_path, 'rb') as file: 
                    att_tuple = pickle.load(file)

                score_dict[f'{tuple_name[:-10]}'] = att_tuple


    convnext_dict = {}
    convnext_dict['img'] = np.array(score_dict[list(score_dict.keys())[1]][0]).squeeze() # not importen which one but I use 1=convnext_rural 

    for i in list(score_dict.keys()):
        if i.split('_')[0] == 'convnext':

            att_name = "_".join(i.split('_')[-2:])
            convnext_dict[att_name] = np.array(score_dict[i][1]).squeeze()

    convnext_df = pd.DataFrame(convnext_dict)
    convnext_df.to_pickle(f"{tuple_dir}/convnext_df.pkl")


    efficientnet_dict = {}
    efficientnet_dict['img'] = np.array(score_dict[list(score_dict.keys())[0]][0]).squeeze() # not importen which one but I use 0= efficientnet rural 

    for i in list(score_dict.keys()):
        if i.split('_')[0] == 'efficientnet':

            att_name = "_".join(i.split('_')[-2:])

            array = np.array(score_dict[i][1]).squeeze()

            norm_array = (array - array.mean())/array.std()

            efficientnet_dict[att_name] = norm_array
        
    efficientnet_df = pd.DataFrame(efficientnet_dict)
    efficientnet_df.to_pickle(f"{tuple_dir}/efficientnet_df.pkl")


    swin_dict = {}
    swin_dict['img'] = np.array(score_dict[list(score_dict.keys())[0]][0]).squeeze() # not importen which one but I use 0= efficientnet rural 

    for i in list(score_dict.keys()):
        if i.split('_')[0] == 'swin':

            att_name = "_".join(i.split('_')[-2:])

            array = np.array(score_dict[i][1]).squeeze()

            #norm_array = (array - array.mean())/array.std()

            swin_dict[att_name] = array#norm_array

    swin_df = pd.DataFrame(swin_dict)
    swin_df.to_pickle(f"{tuple_dir}/swin_df.pkl")  


    return(convnext_df, efficientnet_df, swin_df) 

In [ ]:
convnext_df, efficientnet_df, swin_df = get_ra_dfs()

In [ ]:
# tuple_dir = "/home/simon/Documents/Bodies/data/ra_outputs"

# score_dict = {}

# for root, dirs, files in os.walk(tuple_dir):
#     for tuple_name in files:
#         if tuple_name.split('_')[-1] == 'tuple.pkl':
#             tuple_path = os.path.join(tuple_dir, tuple_name) 
    
#             with open(tuple_path, 'rb') as file: 
#                 att_tuple = pickle.load(file)

#             score_dict[f'{tuple_name[:-10]}'] = att_tuple
            



In [ ]:
# convnext_dict = {}

# convnext_dict['img'] = np.array(score_dict[list(score_dict.keys())[1]][0]).squeeze() # not importen which one but I use 1=convnext_rural 

# for i in list(score_dict.keys()):
#     if i.split('_')[0] == 'convnext':

#         att_name = "_".join(i.split('_')[-2:])
#         convnext_dict[att_name] = np.array(score_dict[i][1]).squeeze()

# convnext_df = pd.DataFrame(convnext_dict)

In [ ]:
# efficientnet_dict = {}

# efficientnet_dict['img'] = np.array(score_dict[list(score_dict.keys())[0]][0]).squeeze() # not importen which one but I use 0= efficientnet rural 

# for i in list(score_dict.keys()):
#     if i.split('_')[0] == 'efficientnet':

#         att_name = "_".join(i.split('_')[-2:])

#         array = np.array(score_dict[i][1]).squeeze()

#         norm_array = (array - array.mean())/array.std()

#         efficientnet_dict[att_name] = norm_array
    
# efficientnet_df = pd.DataFrame(efficientnet_dict)

In [ ]:
# swin_dict = {}

# swin_dict['img'] = np.array(score_dict[list(score_dict.keys())[0]][0]).squeeze() # not importen which one but I use 0= efficientnet rural 

# for i in list(score_dict.keys()):
#     if i.split('_')[0] == 'swin':

#         att_name = "_".join(i.split('_')[-2:])

#         array = np.array(score_dict[i][1]).squeeze()

#         #norm_array = (array - array.mean())/array.std()

#         swin_dict[att_name] = array#norm_array

# swin_df = pd.DataFrame(swin_dict)

In [ ]:
plt.figure(figsize=[25,15])

for i, att in enumerate(list(efficientnet_df.columns)[1:]):

    y = efficientnet_df.loc[:,att]
    x = convnext_df.loc[:,att]

    plt.subplot(2,5, i+1)

    plt.title(att)
    plt.scatter(x,y)

plt.show()

In [ ]:
plt.figure(figsize=[25,15])

for i, att in enumerate(list(swin_df.columns)[1:]):

    y = swin_df.loc[:,att]
    x = convnext_df.loc[:,att]

    plt.subplot(2,5, i+1)

    plt.title(att)
    plt.scatter(x,y)

plt.show()

In [ ]:
def loop_plot(ra_df, model, sample = 'top'):

    for measure in ra_df.columns[1:]: # not img

        n = 16

        if sample == 'top':
            sample_img = ra_df.sort_values(measure, ascending = False)['img'][0:n].values
            scores = ra_df.sort_values(measure, ascending = False)[measure][0:n].values

        elif sample == 'bottom':
            sample_img = ra_df.sort_values(measure, ascending = True)['img'][0:n].values
            scores =ra_df.sort_values(measure, ascending = True)[measure][0:n].values


        elif sample == 'random':
            sample_img = ra_df.sort_values(measure, ascending = True)['img'].sample(n).values
            scores = ra_df.sort_values(measure, ascending = True)[measure].sample(n).values

        else:
            print('wrong input')


        imgs_dir = '/media/simon/Seagate Expansion Drive/images_spanner'
        eks_dir = '/home/simon/Documents/Bodies/figures/ra_eks'

        plt.figure(figsize = [30,20])
        for i, j in enumerate(sample_img):

            path_image = os.path.join(imgs_dir, j)

            plt.subplot(int(np.sqrt(n)), int(np.sqrt(n)), i+1)
            plt.subplots_adjust(hspace = 0.3, wspace = 0.1)
                
            img = cv2.imread(path_image)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # correcting the colors:
            plt.imshow(img)
            plt.title(f'{j}, {measure} = {scores[i]:.3f}')

        suptitle = f'{measure}, {model}, {sample} {n} images'
        plt.suptitle(suptitle, size=16)

        fig_path = f'{eks_dir}/{model}_{measure}_{sample}.pdf'
        plt.savefig(fig_path, bbox_inches="tight")

        plt.show()

In [ ]:
for i in ['top', 'bottom']:
    loop_plot(convnext_df, 'convnext', sample = i)

In [ ]:
for i in ['top', 'bottom']:
    loop_plot(efficientnet_df, 'efficientnet', sample = i)

In [ ]:
for i in ['top', 'bottom']:
    loop_plot(swin_df, 'swin', sample = i)

# OLD

In [ ]:
bodies_pkl = '/home/simon/Documents/Bodies/data/done_dfs/bodies_df_2022_v1_1.pkl'
bodies_df = pd.read_pickle(bodies_pkl)

In [ ]:
output_dir = '/home/simon/Documents/computerome/ra_outputs/'
convnext_pkl = 'swin_t_score_dict' 
efficient_pkl = 'efficientnet_v2_s_score_dict'
swin_pkl = 'swin_t_score_dict'

  
with open(f'{output_dir}{convnext_pkl}.pkl', 'rb') as file: 
    convnext_dict = pickle.load(file)
  
with open(f'{output_dir}{efficient_pkl}.pkl', 'rb') as file: 
    efficient_dict = pickle.load(file)

with open(f'{output_dir}{swin_pkl}.pkl', 'rb') as file: 
    swin_dict = pickle.load(file)


In [ ]:
# just checkin:

print(convnext_dict['all_negative_emotions_t1_id'] == convnext_dict['all_mass_protest_id'])
print(convnext_dict['all_mass_protest_id'] == convnext_dict['all_militarized_id'])
print(convnext_dict['all_militarized_id'] == convnext_dict['all_urban_id'])
print(convnext_dict['all_urban_id'] == convnext_dict['all_negative_emotions_t2_id'])
print(convnext_dict['all_negative_emotions_t2_id'] == convnext_dict['all_privat_id'])
print(convnext_dict['all_privat_id'] == convnext_dict['all_negative_emotions_t1_id'])

print(efficient_dict['all_negative_emotions_t1_id'] == efficient_dict['all_mass_protest_id'])
print(efficient_dict['all_mass_protest_id'] == efficient_dict['all_militarized_id'])
print(efficient_dict['all_militarized_id'] == efficient_dict['all_urban_id'])
print(efficient_dict['all_urban_id'] == efficient_dict['all_negative_emotions_t2_id'])
print(efficient_dict['all_negative_emotions_t2_id'] == efficient_dict['all_privat_id'])
print(efficient_dict['all_privat_id'] == efficient_dict['all_negative_emotions_t1_id'])

print(swin_dict['all_negative_emotions_t1_id'] == swin_dict['all_mass_protest_id'])
print(swin_dict['all_mass_protest_id'] == swin_dict['all_militarized_id'])
print(swin_dict['all_militarized_id'] == swin_dict['all_urban_id'])
print(swin_dict['all_urban_id'] == swin_dict['all_negative_emotions_t2_id'])
print(swin_dict['all_negative_emotions_t2_id'] == swin_dict['all_privat_id'])
print(swin_dict['all_privat_id'] == swin_dict['all_negative_emotions_t1_id'])

print(swin_dict['all_privat_id'] == convnext_dict['all_negative_emotions_t1_id'])
print(convnext_dict['all_privat_id'] == efficient_dict['all_negative_emotions_t1_id'])
# they are all the same


In [ ]:
ra_dict = {}
ra_dict['img'] = convnext_dict['all_negative_emotions_t1_id'] # could be any network and and any id att. They are all the same

for i in list(convnext_dict.keys()):
    if i.endswith('_score'):
        ra_dict[f'convnext_{i}'] = convnext_dict[i][0]

for i in list(efficient_dict.keys()):
    if i.endswith('_score'):
        ra_dict[f'efficient_{i}'] = efficient_dict[i][0]

for i in list(swin_dict.keys()):
    if i.endswith('_score'):
        ra_dict[f'swin_{i}'] = swin_dict[i][0]

ra_df = pd.DataFrame(ra_dict)

In [ ]:
ra_df # lovely but you should not only have 3000 ish images..

In [ ]:
# but you can still see if you think these scores make sense and plot some images...
# or you could corrolate it with some stuff...
# or go on to conflict net....

In [ ]:
ra_df.loc[:,['convnext_all_negative_emotions_t1_score', 'swin_all_negative_emotions_t1_score']]

In [ ]:
ra_df.columns[1:11]

In [ ]:
def loop_plot(sample = 'top'):

    for measure in ra_df.columns[1:11]: # not img and only convnext

        n = 16

        if sample == 'top':
            sample_img = ra_df.sort_values(measure, ascending = False)['img'][0:n].values
            scores = ra_df.sort_values(measure, ascending = False)[measure][0:n].values

        elif sample == 'bottom':
            sample_img = ra_df.sort_values(measure, ascending = True)['img'][0:n].values
            scores =ra_df.sort_values(measure, ascending = True)[measure][0:n].values


        elif sample == 'random':
            sample_img = ra_df.sort_values(measure, ascending = True)['img'].sample(n).values
            scores = ra_df.sort_values(measure, ascending = True)[measure].sample(n).values

        else:
            print('wrong input')


        imgs_dir = '/media/simon/Seagate Expansion Drive/images_spanner'
        eks_dir = '/home/simon/Documents/Bodies/figures/ra_eks'

        plt.figure(figsize = [30,20])
        for i, j in enumerate(sample_img):

            path_image = os.path.join(imgs_dir, j)

            plt.subplot(int(np.sqrt(n)), int(np.sqrt(n)), i+1)
            plt.subplots_adjust(hspace = 0.3, wspace = 0.1)
                
            img = cv2.imread(path_image)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # correcting the colors:
            plt.imshow(img)
            plt.title(f'{j}, {measure} = {scores[i]:.3f}')

        suptitle = f'{measure}, {sample} {n} images'
        plt.suptitle(suptitle, size=16)

        fig_path = f'{eks_dir}/{measure}_{sample}.pdf'
        plt.savefig(fig_path, bbox_inches="tight")

        plt.show()

In [ ]:
for i in ['top', 'bottom']:
    loop_plot(sample = i)

In [ ]:
def plot_sample(dict_of_dfs, att, measure, sample = 'top'):

    n = 16
    #measure = 'ens_score'

    if sample == 'top':
        sample_img = dict_of_dfs[att].sort_values(measure, ascending = False)['img'][0:n].values
        scores = dict_of_dfs[att].sort_values(measure, ascending = False)[measure][0:n].values
    
    elif sample == 'bottom':
        sample_img = dict_of_dfs[att].sort_values(measure, ascending = True)['img'][0:n].values
        scores = dict_of_dfs[att].sort_values(measure, ascending = True)[measure][0:n].values

    
    elif sample == 'radom':
        sample_img = dict_of_dfs[att].sort_values(measure, ascending = True)['img'].sample(n).values
        scores = dict_of_dfs[att].sort_values(measure, ascending = True)[measure].sample(n).values

    else:
        print('wrong input')


    imgs_dir = '/media/simon/Seagate Expansion Drive/images_spanner'

    plt.figure(figsize = [15,10])
    for i, j in enumerate(sample_img):

        path_image = os.path.join(imgs_dir, j)

        plt.subplot(int(np.sqrt(n)), int(np.sqrt(n)), i+1)
        plt.subplots_adjust(hspace = 0.3, wspace = 0.1)
            
        img = cv2.imread(path_image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # correcting the colors:
        plt.imshow(img)
        #plt.xticks([])
        #plt.yticks([])
        plt.title(f'{j}, {measure} = {scores[i]:.3f}')

    suptitle = f'{att}, {sample} {n} images'
    plt.suptitle(suptitle, size=16)

    fig_path = f'plots/{att}_{measure}_{sample}_{n}.pdf'
    plt.savefig(fig_path, bbox_inches="tight")    

    plt.show()

In [ ]:
number_list = []
x = None

while x != 'done':
    x = input()

    try:
        x = float(x)
        number_list.append(x)

    except:

        if x == 'done':
            print('all done...')
            
        
        else:
            print('Not numeric value')
            pass

print(max(number_list))
print(min(number_list))


In [ ]:
while True:
    print(1)

In [ ]:
float('r')

In [ ]:
i_list = [] #  empty list
i_list

In [ ]:

for i in range(10):
    i_list.append(i)

i_list

In [ ]:
i_list += i_list
i_list